In [59]:
from surprise import KNNWithMeans, KNNBasic
from surprise import SVD
from surprise import SVDpp
from surprise import KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

from tqdm import tqdm

import pandas as pd

In [18]:
movies = pd.read_csv('/Users/kseniagorchakova/Library/Mobile Documents/com~apple~CloudDocs/Education/!Profession_DS/Рекомендательные системы/Лекция_3/HW_2/movies.dat', sep="::", names=('movieId', 'title', 'genres')) 
ratings = pd.read_csv('/Users/kseniagorchakova/Library/Mobile Documents/com~apple~CloudDocs/Education/!Profession_DS/Рекомендательные системы/Лекция_3/HW_2/ratings.dat', sep="::", names=('userId', 'movieId', 'rating', 'timestamp'))

<ipython-input-18-4b28818a0fa3>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('/Users/kseniagorchakova/Library/Mobile Documents/com~apple~CloudDocs/Education/!Profession_DS/Рекомендательные системы/Лекция_3/HW_2/movies.dat', sep="::", names=('movieId', 'title', 'genres'))
<ipython-input-18-4b28818a0fa3>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('/Users/kseniagorchakova/Library/Mobile Documents/com~apple~CloudDocs/Education/!Profession_DS/Рекомендательные системы/Лекция_3/HW_2/ratings.dat', sep="::", names=('userId', 'movieId', 'rating', 'timestamp'))


In [28]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [21]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [22]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [23]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),5.0
1,6.0,Toy Story (1995),4.0
2,8.0,Toy Story (1995),4.0
3,9.0,Toy Story (1995),5.0
4,10.0,Toy Story (1995),5.0


In [24]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [25]:
trainset, testset = train_test_split(data, test_size=.15)

### Используем метод KNN

In [30]:
model = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
model.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [32]:
test_pred = model.test(testset)

In [33]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8833


0.883295041306566

In [49]:
# Посчитаем RMSE на основе кросс-валидации
cross_validate(model, data, measures=['RMSE'], cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8899  0.8882  0.8901  0.8889  0.8885  0.8891  0.0008  
Fit time          522.67  528.64  525.79  520.50  514.39  522.40  4.86    
Test time         143.41  135.77  137.13  136.32  136.40  137.81  2.83    


{'test_rmse': array([0.88985743, 0.88822989, 0.89012355, 0.88886861, 0.88846488]),
 'fit_time': (522.6682050228119,
  528.6429500579834,
  525.791081905365,
  520.501825094223,
  514.3947117328644),
 'test_time': (143.40981078147888,
  135.77118730545044,
  137.13365721702576,
  136.3204960823059,
  136.40364480018616)}

### Используем метод KNNBaseline

In [63]:
model_knnbas = KNNBaseline(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
model_knnbas.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [64]:
test_pred_knnbas = model_knnbas.test(testset)

In [65]:
accuracy.rmse(test_pred_knnbas, verbose=True)

RMSE: 0.8636


0.8636230796487963

In [66]:
# Посчитаем RMSE на основе кросс-валидации
cross_validate(model_knnbas, data, measures=['RMSE'], cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8712  0.8676  0.8686  0.8692  0.8663  0.8686  0.0017  
Fit time          479.19  495.49  504.16  506.24  504.10  497.84  10.03   
Test time         174.55  160.58  154.31  151.39  149.27  158.02  9.10    


{'test_rmse': array([0.87124638, 0.86763121, 0.86855945, 0.86923391, 0.86626879]),
 'fit_time': (479.18519830703735,
  495.48685097694397,
  504.16262912750244,
  506.2446479797363,
  504.1012258529663),
 'test_time': (174.54569101333618,
  160.57683873176575,
  154.30526900291443,
  151.38737320899963,
  149.27004098892212)}

### Используем SVD

In [39]:
model_svd = SVD()
model_svd.fit(trainset)

In [40]:
test_pred_svd = model_svd.test(testset)

In [41]:
accuracy.rmse(test_pred_svd, verbose=True)

RMSE: 0.8689


0.8688587902549176

In [48]:
# Посчитаем RMSE на основе кросс-валидации
cross_validate(model_svd, data, measures=['RMSE'], cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8753  0.8756  0.8718  0.8730  0.8750  0.8741  0.0015  
Fit time          68.40   70.38   70.50   70.42   69.77   69.90   0.79    
Test time         2.57    2.11    1.96    1.80    1.69    2.02    0.31    


{'test_rmse': array([0.87529347, 0.87559232, 0.87177941, 0.87300276, 0.87498398]),
 'fit_time': (68.39722323417664,
  70.38440084457397,
  70.50338792800903,
  70.41963911056519,
  69.77361726760864),
 'test_time': (2.5663890838623047,
  2.108466148376465,
  1.9614489078521729,
  1.7966561317443848,
  1.6903679370880127)}

### Используем SVDpp

In [54]:
model_svdpp = SVDpp()
model_svdpp.fit(trainset)

In [58]:
test_pred_svdpp = model_svdpp.test(testset)

In [60]:
accuracy.rmse(test_pred_svdpp, verbose=True)

RMSE: 0.8563


0.85632264516181

In [62]:
# Посчитаем RMSE на основе кросс-валидации
cross_validate(model_svdpp, data, measures=['RMSE'], cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8620  0.8628  0.8635  0.8610  0.8635  0.8625  0.0010  
Fit time          5500.45 5509.08 5524.65 5525.78 5529.63 5517.92 11.20   
Test time         71.52   72.14   64.22   64.92   60.62   66.68   4.45    


{'test_rmse': array([0.86196728, 0.86276438, 0.86354692, 0.86101353, 0.86345633]),
 'fit_time': (5500.44535112381,
  5509.07781291008,
  5524.650968074799,
  5525.783915758133,
  5529.6259329319),
 'test_time': (71.52325201034546,
  72.13640022277832,
  64.22485995292664,
  64.92140197753906,
  60.61508393287659)}